In [6]:
import os
import sys
import warnings

import pandas as pd
from tqdm.notebook import tqdm
import pickle
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names
from functools import partial

from express.datasets import PressingDataset
from tqdm import tqdm
import pandas as pd
from pathlib import Path

import shap
import matplotlib.pyplot as plt


base_path: c:\Users\mirun\24_winter\press\sr-press


In [10]:
model = "xgboost" #"soccermap"
trial = 13 # as you like

path = Path(base_path) / "stores" / "model" / model / f"{trial:03d}" / "component.pkl"
with path.open(mode="rb") as file:
    component = pickle.load(file)
component

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")
test_db = SQLiteDatabase(TEST_DB_PATH)
dataset_test = partial(PressingDataset, path=os.path.join(base_path, "stores", "filtered_datasets", "test"))


In [12]:
print(component.features.keys())
component.test(dataset_test)

dict_keys(['closest_11_players', 'relative_startlocation', 'defenders_in_3m_radius', 'relative_endlocation', 'get_column_sum_to_player'])


{'precision': 0.4098360655737705,
 'recall': 0.08741258741258741,
 'f1': 0.1440922190201729,
 'log_loss': 0.6164863893636872,
 'brier': 0.21067506735907512,
 'roc_auc': 0.5854128411892265}

In [15]:
preds = component.predict(dataset_test)
preds

game_id  action_id
3857254  4            0.116563
         14           0.006936
         17           0.154682
         22           0.300543
         34           0.133272
                        ...   
3869685  3013         0.072382
         3020         0.301446
         3031         0.269038
         3041         0.212329
         3045         0.173190
Length: 14504, dtype: float32

In [16]:
test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"),
    xfns=["closest_11_players", "relative_startlocation", "defenders_in_3m_radius", "relative_endlocation", "get_column_sum_to_player"],
    yfns=["possession_change_by_5_seconds"],
    load_cached=True,
    nb_prev_actions=3
)

### Team_id를 team_name으로 매핑하기

In [8]:
teams_data = []
seen_team_ids = set()
for idx, game in tqdm(test_db.games().iterrows(), total=len(test_db.games())):
    game_id = idx
    
    # game_id에 해당하는 팀 정보를 가져오기
    teams = SBL.teams(game_id)  
    
    # 각 팀에 대해 고유한 팀 정보를 추출
    for i in range(len(teams)):
        team_id = teams.iloc[i]['team_id']
        team_name = teams.iloc[i]['team_name']
        
        # 중복된 team_id는 추가하지 않음
        if team_id not in seen_team_ids:
            teams_data.append({
                'team_id': team_id,
                'team_name': team_name
            })
            seen_team_ids.add(team_id)  # 추적 집합에 추가하여 중복을 방지

teams_df = pd.DataFrame(teams_data)


  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [00:32<00:00,  1.99it/s]


30초 정도 걸리지만 teams_2_id.json 파일을 쓰셔도 됩니다.

In [52]:
import json
team_mapping_dict = teams_df.set_index('team_id')['team_name'].to_dict()

# JSON 파일로 저장
with open('teams_2_id.json', 'w') as file:
    json.dump(team_mapping_dict, file)

In [26]:
players_data = []
seen_player_ids = set()
for idx, game in tqdm(test_db.games().iterrows(), total=len(test_db.games())):
    game_id = idx
    
    # game_id에 해당하는 팀 정보를 가져오기
    players = SBL.players(game_id)  
    
    # 각 팀에 대해 고유한 팀 정보를 추출
    for i in range(len(players)):
        player_id = players.iloc[i]['player_id']
        player_name = players.iloc[i]['player_name']
        
        # 중복된 team_id는 추가하지 않음
        if player_id not in seen_player_ids:
            players_data.append({
                'player_id': player_id,
                'player_name': player_name
            })
            seen_player_ids.add(player_id)  # 추적 집합에 추가하여 중복을 방지

players_df = pd.DataFrame(players_data)


100%|██████████| 64/64 [01:58<00:00,  1.85s/it]


In [ ]:
import json
players_mapping_dict = players_df.set_index('player_id')['player_name'].to_dict()

# JSON 파일로 저장
with open('players_2_id.json', 'w') as file:
    json.dump(players_mapping_dict, file)

### Preds값 가져오기

In [17]:
preds = component.predict(dataset_test)
preds

game_id  action_id
3857254  4            0.116563
         14           0.006936
         17           0.154682
         22           0.300543
         34           0.133272
                        ...   
3869685  3013         0.072382
         3020         0.301446
         3031         0.269038
         3041         0.212329
         3045         0.173190
Length: 14504, dtype: float32

In [22]:
new=test_dataset.labels.copy()
new[f"{component.label[0]}_prob"] = preds.values
new


possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  
game_id action_id                                       
3857254 4                                     0.116563  
        14                                    0.006936  
        17                                    0.154682  
        22                                    0.300543  
        34                                    0.133272  
...                                                ...  
3869685 3013                                  0.072382  
        3020                                  0.301446  
        3031                                  0.269038  
        3041                                  0.212329  
        3045                                  0.173190  

[14504 rows x 2 columns]

### pressing action마다 team_id 찾아서 append하기
- 이것보다 빠르게 구할 수 있는 방법 아신다면 수정해주십시오...
- 50분정도 걸립니다

In [ ]:
from tqdm import tqdm
teams_id = []  # 딕셔너리가 아닌 리스트로 초기화합니다.

# 데이터프레임을 순회하며 각 (game_id, action_id)에 대해 팀 ID를 찾습니다.
for (game_id, action_id), row in tqdm(new.iterrows(), total=len(new)):
   
    # add_names()를 사용하여 특정 game_id에 대한 액션을 가져옵니다.
    actions = add_names(test_db.actions(game_id))

    # 해당 game_id와 action_id를 사용하여 필요한 인덱스의 값을 가져옵니다.
    idx = (game_id, action_id)

    try:
        # 인덱스가 존재할 경우 team_id를 teams_id 리스트에 추가합니다.
        team_id = actions.loc[idx]['team_id']
        teams_id.append(team_id)
    except KeyError:
        # 만약 해당 인덱스가 존재하지 않는다면, None 또는 다른 기본값을 추가합니다.
        teams_id.append(None)


new['team'] = teams_id
new

100%|██████████| 14504/14504 [47:24<00:00,  5.10it/s]


In [105]:
new_player.to_csv('WC_test_player_id.csv', index=False)

WC_test_team_id.csv로 이미 만들어놨습니다.

In [23]:
loaded_teams_df=pd.read_csv('WC_test_team_id.csv')
new['team_id']=loaded_teams_df['team_id'].values
new

possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  team_id  
game_id action_id                                                
3857254 4                                     0.116563      777  
        14                                    0.006936      777  
        17                                    0.154682      777  
        22                                    0.300543      777  
        34                                    0.133272      776  
...                                                ...      ...  
3869685 3013                                  0.072382      771  
        3020                                  0.301446      771  
        3031                                  0.269038      779  
        3041                                  0.212329      779  
        3045                                  0.173190      779  

[14504 rows x 3 columns]

### team_name 매핑해서 append하기

In [29]:
from tqdm import tqdm
import pandas as pd

# 모든 게임의 type_name 데이터를 저장할 리스트 초기화
result_name = []

# 각 게임 ID별로 처리
for game_id in tqdm(test_db.games().index.unique().tolist(), desc="Processing games"):
    # 특정 게임 ID의 액션 데이터를 가져오기
    actions = add_names(test_db.actions(game_id))
    actions = actions[actions['type_name'] == 'pressing']
    
    # result_name 열을 리스트에 추가 (인덱스 유지)
    result_name.append(actions[['result_name']])

# 모든 게임 데이터 통합 (인덱스 유지)
merged_type_names = pd.concat(result_name)

# 결과 확인
print(merged_type_names.head())


Processing games: 100%|██████████| 64/64 [00:08<00:00,  7.68it/s]

                  result_name
game_id action_id            
3857254 4                fail
        14               fail
        17               fail
        22               fail
        25               fail


In [30]:
import json
with open('teams_2_id.json', 'r', encoding='utf-8') as file:
    team_mapping = json.load(file)
new['team_name'] = new['team_id'].astype(str).map(team_mapping)
new['xP_difference'] =  new[f"{component.label[0]}"] - new[f"{component.label[0]}_prob"]
new['result_name'] = merged_type_names['result_name']
new


possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  team_id  team_name  \
game_id action_id                                                            
3857254 4                                     0.116563      777    Tunisia   
        14                                    0.006936      777    Tunisia   
        17                                    0.154682      777    Tunisia   
        22                                    0.300543      777    Tunisia   
        34                                    0.133272      776    Denmark   
...                                                ...      ...        ...   
3869685 3013                                  0.072382      771     France   
        3020                                  0.301446      771     France   
        3031                                  0.269038      779  Argentina   
        3041                                  0.212329      779  Argentina   
        3045                                  0.173190      779  Argentina   

                   xP_difference result_name  
game_id action_id                             
3857254 4              -0.116563        fail  
        14             -0.006936        fail  
        17             -0.154682        fail  
        22             -0.300543        fail  
        34             -0.133272        fail  
...                          ...         ...  
3869685 3013           -0.072382     success  
        3020           -0.301446        fail  
        3031           -0.269038        fail  
        3041           -0.212329        fail  
        3045           -0.173190        fail  

[14504 rows x 6 columns]

In [13]:
xp_gain_best=new[new["possession_change_by_seconds"] == 1].sort_values("xP_difference", ascending=False).head(30)
xp_gain_best.to_pickle("xp_gain_best.pkl")

In [14]:
xp_gain_worst=new[new["possession_change_by_seconds"] == 0].sort_values("xP_difference", ascending=True).head(30)
xp_gain_worst.to_pickle("xp_gain_worst.pkl")

In [36]:
minute={
    "Argentina": 690,
    "Australia": 360,
    "Belgium": 270,
    "Brazil": 480,
    "Cameroon": 270,
    "Canada": 270,
    "Costa Rica": 270,
    "Croatia": 690,
    "Denmark": 270,
    "Ecuador": 270,
    "England": 450,
    "France": 660,
    "Germany": 270,
    "Ghana": 270,
    "Iran": 270,
    "Japan": 390,
    "South Korea": 360,
    "Mexico": 270,
    "Morocco": 660,
    "Netherlands": 480,
    "Poland": 360,
    "Portugal": 450,
    "Qatar": 270,
    "Saudi Arabia": 270,
    "Senegal": 360,
    "Serbia": 270,
    "Spain": 390,
    "Switzerland": 360,
    "Tunisia": 270,
    "United States": 360,
    "Uruguay": 270,
    "Wales": 270
}


In [38]:
# 1. 팀당 pressing point 합 구하기
team_sum = new.groupby('team_name')[f"{component.label[0]}_prob"].sum().reset_index()
# 2. 팀당 xP_difference 합 구하기
team_difference = new.groupby('team_name')['xP_difference'].sum().reset_index()

# 3. 팀당 서로 다른 game_id 개수 세기
team_game_count = new.reset_index().groupby('team_name')['game_id'].nunique()
team_game_count_df = team_game_count.reset_index()


# 4. 팀당 서로 다른 action_id 개수 세기
team_action_count = new.reset_index().groupby('team_name')['action_id'].nunique()
team_action_count= team_action_count.reset_index()

# 데이터프레임 병합

merged_df = pd.merge(team_sum, team_difference, on='team_name', how='outer')  # 첫 번째 병합
merged_df = pd.merge(merged_df, team_game_count, on='team_name', how='outer')  # 두 번째 병합
merged_df = pd.merge(merged_df, team_action_count, on='team_name', how='outer')  # 세 번째 병합
merged_df.rename(columns={'game_id': 'games_played'}, inplace=True)
merged_df.rename(columns={'action_id': 'actions'}, inplace=True)
merged_df.rename(columns={f"{component.label[0]}_prob": 'xP_total'}, inplace=True)

minutes_df = pd.DataFrame(list(minute.items()), columns=['team_name', 'minutes_played'])

# 기존 데이터프레임에 minutes 데이터를 병합
merged_df = pd.merge(merged_df, minutes_df, on='team_name', how='left')

# 추가 컬럼 계산
merged_df['xP_total']=merged_df['xP_total'].round(2)
merged_df['xP_difference']=merged_df['xP_difference'].round(2)
merged_df['xP_per_game'] = (merged_df['xP_total'] / merged_df['games_played']).round(2)
merged_df['xP_per_action'] = (merged_df['xP_total'] / merged_df['actions']).round(2)

merged_df['xP_per_90'] = (merged_df['xP_total'] / (merged_df['minutes_played'] / 90)).round(2)
merged_df['xP_difference_per_90'] = (merged_df['xP_difference'] / (merged_df['minutes_played'] / 90)).round(2)

# 5. 팀당 result_name별 횟수 세기
team_result_counts = new.groupby(['team_name', 'result_name']).size().unstack(fill_value=0).reset_index()

# result_name 열 중 성공과 실패를 분리
team_result_counts.rename(columns={'success': 'success_count'}, inplace=True)

# 성공 횟수 병합
merged_df = pd.merge(merged_df, team_result_counts[['team_name', 'success_count']], on='team_name', how='left')

# 추가 컬럼 계산
merged_df['success_count'] = merged_df['success_count'].fillna(0).astype(int)
merged_df['succes_rate'] = (merged_df['success_count'] / merged_df['actions']).round(2)
column_order = [
    'team_name',               # 팀 이름
    'actions',                 # 총 action_id 수    
    'xP_total',                # 팀당 pressing point 합
    'xP_difference',           # xP_difference
    'minutes_played',          # 총 플레이 시간 (분)
    'xP_per_game',             # 경기당 xP
    'xP_per_action',           # action당 xP
    'xP_per_90',               # 90분당 xP
    'xP_difference_per_90',    # 90분당 xP_difference
    'success_count',            # 성공 횟수
    'succes_rate'              # 성공률
]

# 열 순서를 재배치
merged_df = merged_df[column_order]

# 정렬
merged_df.sort_values('xP_difference_per_90', ascending=False, inplace=True)

merged_df

,team_name,actions,xP_total,xP_difference,minutes_played,xP_per_game,xP_per_action,xP_per_90,xP_difference_per_90,success_count,succes_rate
30,Uruguay,315,89.110001,28.889999,270,29.70,0.28,29.70,9.63,65,0.21
26,Spain,380,121.269997,39.730000,390,30.32,0.32,27.99,9.17,133,0.35
8,Denmark,274,78.059998,21.940001,270,26.02,0.28,26.02,7.31,53,0.19
16,Mexico,317,93.669998,21.330000,270,31.22,0.30,31.22,7.11,64,0.20
31,Wales,317,86.459999,19.540001,270,28.82,0.27,28.82,6.51,59,0.19
3,Brazil,480,147.300003,33.700001,480,29.46,0.31,27.62,6.32,121,0.25
18,Netherlands,583,171.979996,31.020000,480,34.40,0.29,32.25,5.82,104,0.18
9,Ecuador,353,99.820000,16.180000,270,33.27,0.28,33.27,5.39,79,0.22
11,France,682,195.690002,35.310001,660,27.96,0.29,26.69,4.82,132,0.19
17,Morocco,811,237.449997,33.549999,660,33.92,0.29,32.38,4.57,182,0.22
